In [1]:
import pandas as pd
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk import ngrams
from langdetect import detect

In [2]:
#input gold price dataset and convert it into time series data
rawPriceData=pd.read_csv('/Users/Mandy/Study/SpringBoard/Capstone 2/GoldPriceData-Raw.csv')
rawPriceData['Date'] = pd.to_datetime(rawPriceData['Date'])
rawPriceData.set_index(rawPriceData['Date'],inplace=True)

In [3]:
# filling weekend/public holiday missing data by taking average.
priceData=rawPriceData.resample('D').interpolate()
priceData=priceData['Price']

In [4]:
#input tweet dataset
rawTweetData=pd.read_csv('/Users/Mandy/Study/SpringBoard/Capstone 2/GoldTweetDataWorkBook.csv',encoding='latin1')
print(rawTweetData.head())

      date  score  language                                               text
0  3/31/18      0       NaN  à¸à¸±à¸à¸à¸µà¸à¸µà¹à¸à¹à¸²à¸§à¸à¸£à¸¡à...
1  3/31/18     -1       NaN               Donald Trump and Roseanne are living
2  3/31/18     -1       NaN                              I know we're all numb
3  3/31/18     -1       NaN                              I know we're all numb
4  3/31/18      1       NaN  Diaspora culture is thinking you know better t...


In [7]:
#detect non-English tweets and delete those rows
for r in rawTweetData.index:
    eachRowText=rawTweetData.text[r]
    rawTweetData.loc[r,2]=detect(eachRowText)

print(rawTweetData.language[0])
print(rawTweetData.language[1])
print(rawTweetData.language[2])

rawTweetData=rawTweetData[rawTweetData.language=='en']

LangDetectException: No features in text.

In [8]:
#replace special characters, such as URLs, usernames, hashtags, pictures with"URL","USER","HASHTAG","PICTURE"
#also remove all the special characters
for row in rawTweetData.index:
    eachTweet=rawTweetData.text[row]
    eachTweet_url=re.sub(r"https:// \S+","URL",eachTweet)
    eachTweet_pic=re.sub(r"pic.\S+","PICTURE",eachTweet_url)
    eachTweet_user=re.sub(r"@\S+","USER",eachTweet_pic)
    eachTweet_hashtag=re.sub(r"#\S+","HASHTAG",eachTweet_user)
    eachTweet_spechar=re.sub("[^A-Za-z0-9]+"," ",eachTweet_hashtag)
    rawTweetData.set_value(row,'Tweet',eachTweet_spechar)
print(rawTweetData.text[0])
print(rawTweetData.text[1])
print(rawTweetData.text[2])
print(rawTweetData.text[3])
print(rawTweetData.text[4])

à¸à¸±à¸à¸à¸µà¸à¸µà¹à¸à¹à¸²à¸§à¸à¸£à¸¡à¸­à¸¸à¸à¸¢à¸²à¸à¹à¸à¸£à¸µà¸¢à¸¡à¸à¸´à¸à¸­à¹à¸²à¸§à¸¡à¸²à¸«à¸¢à¸²à¸à¸£à¸°à¸à¸²à¸¢à¸­à¸­à¸à¹à¸ à¸à¸±à¹à¸à¹à¸¥à¸à¸à¹à¸²à¸à¸à¸²à¸à¸£à¸±à¸ à¸ªà¸³à¸à¸±à¸à¸à¹à¸²à¸§à¸à¸±à¹à¸§à¹à¸¥à¸à¸¥à¸à¸à¹à¸²à¸§à¹à¸£à¸·à¹à¸­à¸à¸à¸µà¹à¸¡à¸²à¸à¸¡à¸²à¸¢ à¹à¸­à¸à¸µ - âà¸à¹à¸²à¹à¸£à¸²à¹à¸¡à¹à¸à¸³à¹à¸à¸§à¸±à¸à¸à¸µà¹ à¸¡à¸±à¸à¸à¸°à¸ªà¸²à¸¢à¹à¸à¸´à¸à¹à¸â (à¸à¸³à¹à¸«à¹à¸ªà¸±à¸¡à¸ à¸²à¸©à¸à¹à¸à¸­à¸à¸à¹à¸²à¸à¸­à¸à¸´à¸à¸à¸µà¸à¸£à¸¡à¸­à¸¸à¸à¸¢à¸²à¸) world economic forum - à¸à¸£à¸°à¹à¸à¸¨à¹à¸à¸¢à¸à¸³à¸¥à¸±à¸à¸à¸°à¸à¸´à¸à¸«à¸²à¸à¸ªà¸§à¸¢à¸à¸µà¹à¸ªà¸¸à¸à¹à¸à¹à¸¥à¸ à¹à¸à¸·à¹à¸­à¸à¸·à¹à¸à¸à¸·à¸à¸à¸£à¸£à¸¡à¸à¸²à¸à¸´ pic.twitter.com/Hm75PKAZh4
Donald Trump and Roseanne are living
I know we're all numb
I know we're all numb
Diaspora culture is thinking you know better than all the people who stayed home combined and thinking you will go back home and change the culture/ econo

In [9]:
#Split tweet text word by word (tokenize)
tokenizedTweet = rawTweetData.apply(lambda row: word_tokenize(row['text']), axis=1)
print(tokenizedTweet.head())

0    [à¸à¸±à¸à¸à¸µà¸à¸µà¹à¸à¹à¸²à¸§à¸à¸£à¸¡...
1          [Donald, Trump, and, Roseanne, are, living]
2                        [I, know, we, 're, all, numb]
3                        [I, know, we, 're, all, numb]
4    [Diaspora, culture, is, thinking, you, know, b...
dtype: object


In [10]:
#remove stopwords( words that don't have any positive/negative meanings)
filteredTweet = tokenizedTweet.apply(lambda x: [word for word in x if word not in stopwords.words('english')])
print(filteredTweet.head())

0    [à¸à¸±à¸à¸à¸µà¸à¸µà¹à¸à¹à¸²à¸§à¸à¸£à¸¡...
1                    [Donald, Trump, Roseanne, living]
2                                 [I, know, 're, numb]
3                                 [I, know, 're, numb]
4    [Diaspora, culture, thinking, know, better, pe...
dtype: object
